# AWS Strands Agent with ComfyUI MCP Server Example

This notebook demonstrates how to use AWS Strands Agent to call the ComfyUI MCP Server for image and video generation.

## Prerequisites

1. ComfyUI server running on localhost:8188
2. Required Python packages installed
3. MCP server script available

## Setup

In [ ]:
import os
import json
import asyncio
import subprocess
from pathlib import Path
from strands import Agent
from strands.tools.mcp import MCPTool
import tempfile
from IPython.display import Image, Video, display
import base64

## Configure Environment Variables

In [ ]:
# Set ComfyUI server URL
os.environ['COMFYUI_SERVER_URL'] = 'http://localhost:8188'
os.environ['COMFYUI_ENABLE_FALLBACK'] = 'true'

# Create output directory
output_dir = Path('./generated_outputs')
output_dir.mkdir(exist_ok=True)

print(f"Output directory: {output_dir.absolute()}")

## Initialize MCP Tool and Agent

In [ ]:
# Path to the MCP server script
mcp_server_path = Path('./mcp_server_stdio_sample.py').absolute()

# Create MCP tool using FastMCP
mcp_tool = MCPTool(
    name="comfyui_generator",
    command=["python", str(mcp_server_path)],
    description="ComfyUI image and video generation tools built with FastMCP"
)

# Initialize Strands Agent with MCP tool
agent = Agent(
    name="ComfyUI Generator Agent",
    instructions="You are an AI assistant that can generate images and videos using ComfyUI. You can create images from text descriptions, transform existing images, and generate videos from text or images.",
    tools=[mcp_tool]
)

print("Agent initialized with ComfyUI MCP tools")

## Example 1: Text-to-Image Generation

In [ ]:
# Generate an image from text
text_to_image_prompt = """
Generate a beautiful landscape image with the following description:
"A serene mountain lake at sunset with snow-capped peaks reflected in the calm water, surrounded by pine trees"

Save the generated image to './generated_outputs/mountain_lake_sunset.png'
"""

response = await agent.run(text_to_image_prompt)
print("Agent Response:")
print(response.content)

# Display the generated image if it exists
image_path = output_dir / 'mountain_lake_sunset.png'
if image_path.exists():
    display(Image(filename=str(image_path)))
    print(f"\nImage saved to: {image_path}")
else:
    print("Image file not found. Check the agent response for details.")

## Example 2: Image-to-Image Transformation

In [ ]:
# First, let's use the previously generated image (if available) or create a sample image
input_image_path = output_dir / 'mountain_lake_sunset.png'

if input_image_path.exists():
    image_to_image_prompt = f"""
    Transform the image at '{input_image_path}' with the following description:
    "Convert this landscape into a winter scene with snow falling and frozen lake"
    
    Save the transformed image to './generated_outputs/winter_mountain_lake.png'
    """
    
    response = await agent.run(image_to_image_prompt)
    print("Agent Response:")
    print(response.content)
    
    # Display the transformed image
    output_image_path = output_dir / 'winter_mountain_lake.png'
    if output_image_path.exists():
        print("\nOriginal Image:")
        display(Image(filename=str(input_image_path)))
        print("\nTransformed Image:")
        display(Image(filename=str(output_image_path)))
        print(f"\nTransformed image saved to: {output_image_path}")
    else:
        print("Transformed image file not found. Check the agent response for details.")
else:
    print("Input image not found. Please run the text-to-image example first.")

## Example 3: Text-to-Video Generation

In [ ]:
# Generate a video from text
text_to_video_prompt = """
Generate a short video with the following description:
"A peaceful ocean wave gently rolling onto a sandy beach at golden hour"

Save the generated video to './generated_outputs/ocean_waves.mp4'
Use 16 fps and 15 steps for faster generation.
"""

response = await agent.run(text_to_video_prompt)
print("Agent Response:")
print(response.content)

# Display the generated video if it exists
video_path = output_dir / 'ocean_waves.mp4'
if video_path.exists():
    display(Video(str(video_path), width=512, height=512))
    print(f"\nVideo saved to: {video_path}")
else:
    print("Video file not found. Check the agent response for details.")

## Example 4: Image-to-Video Generation

In [ ]:
# Use an existing image to generate a video
input_image_path = output_dir / 'mountain_lake_sunset.png'

if input_image_path.exists():
    image_to_video_prompt = f"""
    Create a video animation from the image at '{input_image_path}' with the following description:
    "Animate this mountain lake scene with gentle water ripples and moving clouds in the sky"
    
    Save the generated video to './generated_outputs/animated_mountain_lake.mp4'
    Use 16 fps and 15 steps for faster generation.
    """
    
    response = await agent.run(image_to_video_prompt)
    print("Agent Response:")
    print(response.content)
    
    # Display the source image and generated video
    video_path = output_dir / 'animated_mountain_lake.mp4'
    if video_path.exists():
        print("\nSource Image:")
        display(Image(filename=str(input_image_path)))
        print("\nGenerated Video:")
        display(Video(str(video_path), width=512, height=512))
        print(f"\nVideo saved to: {video_path}")
    else:
        print("Video file not found. Check the agent response for details.")
else:
    print("Input image not found. Please run the text-to-image example first.")

## Example 5: Batch Generation with Custom Parameters

In [ ]:
# Generate multiple images with different styles
batch_prompt = """
Generate three different style variations of a cat sitting on a windowsill:

1. Realistic style - save to './generated_outputs/cat_realistic.png' with high quality settings (30 steps, cfg_scale 8.0)
2. Cartoon style - save to './generated_outputs/cat_cartoon.png' with balanced settings (20 steps, cfg_scale 7.0)
3. Oil painting style - save to './generated_outputs/cat_painting.png' with artistic settings (25 steps, cfg_scale 7.5)

Use the same seed (12345) for all generations to ensure consistency in composition.
"""

response = await agent.run(batch_prompt)
print("Agent Response:")
print(response.content)

# Display all generated images
image_files = ['cat_realistic.png', 'cat_cartoon.png', 'cat_painting.png']
for img_file in image_files:
    img_path = output_dir / img_file
    if img_path.exists():
        print(f"\n{img_file}:")
        display(Image(filename=str(img_path)))
    else:
        print(f"\n{img_file}: Not found")

## Utility Functions

In [ ]:
def list_generated_files():
    """List all generated files in the output directory"""
    print(f"Generated files in {output_dir}:")
    for file_path in output_dir.iterdir():
        if file_path.is_file():
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"  {file_path.name} ({size_mb:.2f} MB)")

def cleanup_generated_files():
    """Clean up all generated files"""
    for file_path in output_dir.iterdir():
        if file_path.is_file():
            file_path.unlink()
            print(f"Deleted: {file_path.name}")

# List current files
list_generated_files()

## Troubleshooting

If you encounter issues:

1. **ComfyUI Server Not Running**: Make sure ComfyUI is running on localhost:8188
2. **Missing Models**: Ensure required models are installed in ComfyUI
3. **Permission Errors**: Check file permissions for the output directory
4. **Memory Issues**: Reduce image dimensions or use fewer sampling steps

Check the agent responses for detailed error messages.